# Projeto 2 - Classificador Automático de Sentimento

Professor: Raul Ikeda

Alunos: Rafael da Fonte e Eduardo Cunha

___
## Programa


In [170]:
import pandas as pd
import numpy as np

# Importando os dois dataframes

excel = pd.ExcelFile('tweets_airpods_201809062235.xlsx')
data_treinamento = pd.read_excel(excel, 'Treinamento')
data_teste = pd.read_excel(excel, 'Teste')

# Limpando as colunas Treinamento e Teste dos dataframes

treinamento = data_treinamento['Treinamento'].str.replace('@', '')
treinamento = treinamento.str.replace('rt', '')
treinamento = treinamento.str.replace(',', '')
treinamento = treinamento.str.replace('.', '')
treinamento = treinamento.str.replace(':', '')
treinamento = treinamento.str.replace(';', '')
treinamento = treinamento.str.replace('\n', '')
treinamento = treinamento.str.replace('"', '')
treinamento = treinamento.str.replace('“', '')
treinamento = treinamento.str.replace('”', '')
treinamento = treinamento.str.replace('$', ' $ ')
treinamento = treinamento.str.replace('!', ' ! ')
treinamento = treinamento.str.replace('?', ' ? ')

teste = data_teste['Teste'].str.replace('@', '')
teste = teste.str.replace('rt', '')
teste = teste.str.replace(',', '')
teste = teste.str.replace('.', '')
teste = teste.str.replace(':', '')
teste = teste.str.replace(';', '')
teste = teste.str.replace('\n', '')
teste = teste.str.replace('"', '')
teste = teste.str.replace('“', '')
teste = teste.str.replace('”', '')
teste = teste.str.replace('“', '')
teste = teste.str.replace('”', '')
teste = teste.str.replace('$', ' $ ')
teste = teste.str.replace('!', ' ! ')
teste = teste.str.replace('?', ' ? ')

# Inserindo as colunas limpas em seus respectivos dataframes

data_treinamento['Treinamento'] = treinamento
data_teste['Teste'] = teste

# Selecionando os relevantes e irrelevantes da base de treinamento

Relevantes = data_treinamento[data_treinamento['PRODUTO : APPLE AIRPODS'] == 'Relevante']
Irrelevantes = data_treinamento[data_treinamento['PRODUTO : APPLE AIRPODS'] == 'Irrelevante']

# Definindo P(Relevante) e P(Irrelevante)

PRelevante = len(Relevantes)/len(data_treinamento)
PIrrelevante = len(Irrelevantes)/len(data_treinamento)

# Comparando P(Relevante|tweet) com P(Irrelevante|tweet)
# P(Relevante|tweet) = P(tweet|Relevante) x P(Relevante)
# P(Irrelevante|tweet) = P(tweet|Irrelevante) x P(Irrelevante)

# Criando as listas com tweets, sendo cada um uma lista com as palavras separadas

lista_tweets = []
lista_tweets_relevantes = []
lista_tweets_irrelevantes = []

for tweet in Relevantes['Treinamento']:
    lista_tweets_relevantes.append(tweet.split())
    
for tweet in Irrelevantes['Treinamento']:
    lista_tweets_irrelevantes.append(tweet.split())

for tweet in treinamento:
    lista_tweets.append(tweet.split())
    
# Verificando palavras possiveis para Laplace Smoothing

palavras_possiveis = []

for tweet in lista_tweets:
    for palavra in tweet:
        if palavra.lower() not in palavras_possiveis:
            palavras_possiveis.append(palavra.lower())
            
# Calcula P(palavra|Relevante) com Laplace Smoothing    
    
def ProbRelevante(palavra):
    ocorrencia = 1
    total_de_palavras = 0
    
    for tweet in lista_tweets_relevantes:
        total_de_palavras += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                ocorrencia += 1
                
    return ocorrencia/(total_de_palavras + len(palavras_possiveis))

# Calcula P(palavra|Irrelevante) com Laplace Smoothing

def ProbIrrelevante(palavra):
    ocorrencia = 1
    total_de_palavras = 0
    
    for tweet in lista_tweets_irrelevantes:
        total_de_palavras += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                ocorrencia += 1
    
    return ocorrencia/(total_de_palavras + len(palavras_possiveis))

# Classificando os tweets da base de treinamento

def classifica(lista_de_tweets):
    classificacao = []

    for tweet in lista_de_tweets: 
        tweet_s = tweet.split()
    
        PPRelevantes = []            
        PPIrrelevantes = []
        for palavra in tweet_s:
            if palavra[0:5] != 'https':
                PPRelevantes.append(ProbRelevante(palavra))
                PPIrrelevantes.append(ProbIrrelevante(palavra))

        PTweetRelevante = 1
        PTweetIrrelevante = 1

        for i in PPRelevantes:
            PTweetRelevante *= i

        for i in PPIrrelevantes:
            PTweetIrrelevante *= i

        PRelevanteTweet = PTweetRelevante * PRelevante
        PIrrelevanteTweet = PTweetIrrelevante * PIrrelevante

        if PRelevanteTweet > PIrrelevanteTweet:
            classificacao.append('Relevante')
        else:
            classificacao.append('Irrelevante')
    return classificacao

lista_tweets_teste = []
for tweet in teste:
    lista_tweets_teste.append(tweet)

classificacao = classifica(lista_tweets_teste)

gabarito = data_teste.rename(index=str, columns={'Unnamed: 1':'Relevancia'}).Relevancia

___
## Verificando a performance


In [177]:
falpos = 0
falneg = 0
verpos = 0
verneg = 0

for i in range(len(gabarito)):
    if gabarito[i] == classificacao[i] and gabarito[i] == 'Relevante':
        verpos += 1
    elif gabarito[i] == classificacao[i] and gabarito[i] == 'Irrelevante':
        verneg += 1
    elif gabarito[i] != classificacao[i] and gabarito[i] == 'Relevante' and classificacao[i] == 'Irrelevante':
        falneg += 1
    else:
        falpos += 1

tweets_classificados_relevantes = []

for i in range(len(classificacao)):
    if classificacao[i] == "Relevante":
        tweets_classificados_relevantes.append(lista_tweets_teste[i])
        
falpos /= len(classificacao)
falneg /= len(classificacao)
verpos /= len(classificacao)
verneg /= len(classificacao)

#print(verpos+verneg+falpos+falneg)

print("Negativo Falso: {}%".format(falneg*100))
print("Positivo Falso: {}%".format(falpos*100))
print("Negativo Verdadeiro: {}%".format(verneg*100))
print("Positivo Verdadeiro: {}%".format(verpos*100))

Negativo Falso: 5.5%
Positivo Falso: 15.5%
Negativo Verdadeiro: 73.0%
Positivo Verdadeiro: 6.0%


## Tweets classificados como relevantes

In [179]:
for i in tweets_classificados_relevantes:
    print(i)

apple airpodsany good ?  these huawei freebuds are cack
after these new airpods / cases drop i'm making it a task to remove all plug in chargers from the crib
20 mos it'd taken to learn what kind of ux was hidden beneath that case o cruel misunderstanding !  o stubborn self-willed exile from the loving breast !  but it was all right everything was all right the struggle was finished he had won the victory over himself he loved airpods https//tco/emqjnipd0c
 tokenasty i just spent 10 minutes looking for my airpods already in my ears wtf
sfmta_muni castro station i dropped my  $  airpods on inbound tracks between 1st/2nd benches  guessing i shouldn't get them myself ?   can u save my day ?   keep losing the damn things argh ! 
it’s hard to keep up with airpods them hoes stay falling out
how you manage work airpods into the tweet bost ma brain 👏🏾 https//tco/hddk6atli2
gorgeousahar you damn right / even though i purchased them once after the main ones broke i knew it was gonna be a bad inv

___
## Concluindo


    O classificador foi capaz de julgar corretamente a relevância de 79% dos tweets. Dos 21% classificados erroneamente, 15.5% foram marcados como relevantes sendo na verdade irrelevantes, enquanto 5.5% foram marcados como irrelevantes sendo na verdade relevantes. Isso significa que caso de fato utilize-se esse classificador para obter informações do produto, 82,5% da informação irrelevante será filtrada corretamente. Em contrapartida, 47,8% dos tweets relevantes serão classificados como irrelevantes. Caso uma equipe de marketing analisasse todos os tweets testados, apenas 11,5% deles seriam relevantes. Com o uso do filtro do classificador, 27,9% dos tweets seriam relevantes. Assim sendo, mesmo que quase metade dos tweets relevantes não sejam detectados pelo classificador, a chance de um tweet ser relevante após a classificação quase triplica. Dessa forma, com o aumento da base de análise, a eficiência da equipe que anaisa os tweets aumenta, tendo em vista que um maior percentual do conteúdo analisado será de fato relevante.
    
    Com relação a mensagens com dupla negação ou sarcasmo, o classficador se mostra ineficiente por tratar cada palavra individualmente e não levar o contexto ou outras palavras do tweet em consideração na avaliação individual.
    Assim como já explanado, após o uso do classificador, a probalbilidade de um tweet a ser analisado pela equipe de marketing ser relevante, aumenta em quase 250%, otimizando assim a análise. Entretanto, viu-se também que grande parte dos tweets relevantes foram perdidos e portanto, uma melhora no programa se mostra necessária. Tal evolução se tornaria possível através do aumento da base de treinamento, tornando mais precisa a probabilidade calculada de cada palavra, bem como a implementação da associação de palavras no cálculo das probabilidades (como por exemplo associar o "não" a palavra seguinte de modo a diferenciar a probabilidade de "não" vezes a de "gosto", da probabiliade de ocorrer "não gosto").
    
    Pode-se argumentar que agora que se possui um classificador funcional, esse programa poderia ser utilizado para aumentar a base de treinamento, supostamente aumentando o índice de acerto. Para explicar porque isso é uma péssima ideia, pode-se fazer uma analogia universitária. Digamos que um aluno do curso de engenharia do Insper decide ver uma palestra sobre contabilidade para engenheiros. Alimentar a base de treinamento com o próprio programa seria como pedir para esse aluno dar uma aula de um semestre para uma sala de alunos do curso de economia. Ou seja, Uma pessoa que sabia pouco sobre o assunto está ensinando a um grupo grande de pessoas que sabem menos ainda. Analogamente, o classificador que já possui um índice de erro grande, irá apenas propagar o erro através dessa estratégia.
    
    